In [ ]:
import tensorflow as tf # tensorflow 2.10
import numpy as np
import matplotlib.pyplot as plt
from load_data2 import evals_parser, evals_square_parser, evecs_parser

In [ ]:
# Importing the datasets
batch_size = 16
tfile = "/home/gaspard/ETHZ/Master_Arbeit/acoupipe_datasets/training_1-100000_csmtriu_1src_he4.0625-1393.4375Hz_ds1-v001_13-Sep-2022.tfrecord"
dataset = tf.data.TFRecordDataset(filenames=[tfile])

evals_dataset = dataset.map(evals_parser)
evals_dataset = evals_dataset.shuffle(buffer_size=10).batch(batch_size)

evals_square_dataset = dataset.map(evals_square_parser)
evals_square_dataset = evals_square_dataset.shuffle(buffer_size=10).batch(batch_size)

evecs_dataset = dataset.map(evecs_parser)
evecs_dataset = evecs_dataset.shuffle(buffer_size=10).batch(batch_size)

### WGANGP to generate eigenvalues

In [ ]:
from evals_WGANGP2 import evals_WGANGP

# Creating GAN
evals_wgangp = evals_WGANGP()

In [ ]:
evals_wgangp.train(evals_dataset=evals_square_dataset, n_epoch=100)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

is_real, sample = evals_wgangp.generate_evals()
sample = np.sort(np.array(sample[0, :, :, 0]).flatten()) # convert sample from (1,8,8,1) to (64,)


if is_real:
    plt.title("generated sample, detected as real")
    plt.plot(sample)
else:
    plt.title("generated sample, detected as fake")
    plt.plot(sample)    

### GRIDSEARCH: GAN to generate eigenvalues

In [ ]:
from tensorflow import summary
from evals_GAN2 import evals_GAN

lr_list = [5e-7, 1e-7, 5e-8, 1e-8, 5e-9, 1e-9]
#lr_list = [1e-10, 1e-11, 1e-12]
beta_1_list = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

for lr in lr_list:
    for beta_1 in beta_1_list:
        
        print(f"lr={lr}, beta_1={beta_1}")
        
        evals_gan = evals_GAN(lr, beta_1, summary=False) # Creating GAN
        evals_gan.train(evals_dataset=evals_dataset, n_epoch=500, print_epoch=False) # Training the GAN

        # Plotting sampled eigenvalues 
        is_real, sample = evals_gan.get_evals()
        
        fig = plt.figure()

        if is_real:
            plt.title(f"generated sample, detected as real, lr={lr}, b={beta_1}")
            plt.plot(np.sort(np.array(sample)[0]))
        else:
            plt.title(f"generated sample, detected as fake, lr={lr}, b={beta_1}")
            plt.plot(np.sort(np.array(sample)[0]))

In [ ]:
# Grid search 2
#from tensorflow import summary
from evals_GAN2 import evals_GAN

lr_list = [1e-9, 1e-9, 1e-8, 1e-7, 1e-7]
beta_1_list = [0.5, 0.7, 0.2, 0.1, 0.9]

n_epoch=250

for lr, beta_1 in zip(lr_list, beta_1_list):
    
    print(f"lr={lr}, beta_1={beta_1}")
    
    evals_gan = evals_GAN(lr, beta_1, summary=False) # Creating GAN
    evals_gan.train(evals_dataset=evals_dataset, n_epoch=n_epoch, print_epoch=False) # Training the GAN

    # Plotting sampled eigenvalues 
    is_real, sample = evals_gan.get_evals()
    
    fig = plt.figure()

    if is_real:
        plt.title(f"generated sample, detected as real, lr={lr}, b={beta_1}")
        plt.plot(np.sort(np.array(sample)[0]))
    else:
        plt.title(f"generated sample, detected as fake, lr={lr}, b={beta_1}")
        plt.plot(np.sort(np.array(sample)[0]))
    
# runtime   : 1000 epochs  : approx 5min30
# runtime   : 250 epochs   : approx 1min30

### GAN to generate eigenvalues

In [ ]:
from evals_GAN2 import evals_GAN

# Creating GAN
evals_gan = evals_GAN()

# Training the GAN
evals_gan.train(evals_dataset=evals_dataset, n_epoch=100)

In [ ]:
# Plotting sampled eigenvalues 
is_real, sample = evals_gan.get_evals()

if is_real:
    plt.title("generated sample, detected as real")
    plt.plot(np.sort(np.array(sample)[0]))
else:
    plt.title("generated sample, detected as fake")
    plt.plot(np.sort(np.array(sample)[0]))

### DCGAN to generate eigenvalues

In [ ]:
from evals_DCGAN2 import evals_DCGAN

# Creating GAN
evals_dcgan = evals_DCGAN()

# Training the GAN
evals_dcgan.train(evals_dataset=evals_dataset, n_epoch=300)

In [ ]:
# Plotting sampled eigenvalues 
is_real, sample = evals_dcgan.get_evals()

if is_real:
    plt.title("generated sample, detected as real")
    plt.plot(np.sort(np.array(sample)[0]))
else:
    plt.title("generated sample, detected as fake")
    plt.plot(np.sort(np.array(sample)[0]))

### WGAN to generate eigenvectors 

In [ ]:
from evecs_WGAN2 import evecs_WGAN

# Creating WGAN
evecs_wgan = evecs_WGAN()

In [ ]:
evecs_wgan.train(evecs_dataset=evecs_dataset, n_epoch=1000)

In [ ]:
sample = evecs_wgan.get_evecs()